# V3 Fraction Collector (Runze Valve)

In [1]:
from fraction_collector_v4 import FractionCollector
from sample_producer_flowreactor import SampleProducer
import threading
import queue
import time

In [2]:
# Initialize the FractionCollector
collector = FractionCollector(sensor_id=1, runze_valve_port='COM9', runze_valve_address=0, runze_valve_num_port=10, collection_num=3, waste_num=6)

Connected to CNC Machine!
Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']


In [ ]:
#collector.set_valve_state(3)

Current port set to 3


In [3]:
# Define well locations
loc = 'well_plate_location'
#loc = '96_well_plate_location'

In [ ]:
# Shared queue and event
sample_queue = queue.Queue()
stop_event = threading.Event()
number_of_samples = 1

# Initialize producer
producer = SampleProducer(
    reaction_profile_csv='HCW8.csv',
    sample_queue=sample_queue,
    stop_event=stop_event,
    tubing_length_m=1.0,
    tubing_id_inch=0.03,
    buffer_factor=1.2
)
producer.start()

[14:21:50] === Sample Collection Summary ===
[14:21:50] Reaction name         : HCW8
[14:21:50] Waste time (s)        : 1514.1
[14:21:50] Collection time (s)   : 1026.6
[14:21:50] Flow rate (mL/min)    : 0.50
[14:21:50] Tubing length (m)     : 1.0
[14:21:50] Tubing ID (inch)      : 0.03
[14:21:50] Buffer factor         : 1.2
[14:21:50] Tubing delay (s)      : 54.7
[14:21:50] Adjusted waste time   : 1579.8 s
[14:21:50] Adjusted collect time : 855.5 s
[14:21:50] =================================
[14:21:50] Starting waste phase for 1579.8 seconds...


[14:22:20] Wasting... 30 / 1579 seconds elapsed
[14:22:50] Wasting... 60 / 1579 seconds elapsed
[14:23:20] Wasting... 90 / 1579 seconds elapsed
[14:23:50] Wasting... 120 / 1579 seconds elapsed
[14:24:20] Wasting... 150 / 1579 seconds elapsed
[14:24:50] Wasting... 180 / 1579 seconds elapsed
[14:25:20] Wasting... 210 / 1579 seconds elapsed
[14:25:50] Wasting... 240 / 1579 seconds elapsed
[14:26:20] Wasting... 270 / 1579 seconds elapsed
[14:26:50] Wasting... 300 / 1579 seconds elapsed
[14:27:20] Wasting... 330 / 1579 seconds elapsed
[14:27:50] Wasting... 360 / 1579 seconds elapsed
[14:28:20] Wasting... 390 / 1579 seconds elapsed
[14:28:50] Wasting... 420 / 1579 seconds elapsed
[14:29:20] Wasting... 450 / 1579 seconds elapsed
[14:29:50] Wasting... 480 / 1579 seconds elapsed
[14:30:20] Wasting... 510 / 1579 seconds elapsed
[14:30:50] Wasting... 540 / 1579 seconds elapsed
[14:31:20] Wasting... 570 / 1579 seconds elapsed
[14:31:50] Wasting... 600 / 1579 seconds elapsed
[14:32:20] Wasting... 6

In [ ]:
# Simulate collection
loc_index = 0
loc_threshold = loc_index + number_of_samples
collection_thread = None

while not stop_event.is_set():
    try:
        sample = sample_queue.get(timeout=0.1)
        if collection_thread and collection_thread.is_alive():
            print("❌ Not enough time between samples — shutting down.")
            stop_event.set()
            break

        #collection_thread = threading.Thread(target=collect_sample_sim, args=(sample, loc_index, 3))
        collection_thread = threading.Thread(target=collector.collect_fraction, args=(70, loc, loc_index, 20)) #real collection
        collection_thread.start()
        loc_index += 1

    except queue.Empty:
        pass

    if loc_index >= loc_threshold:  # match number of samples
        break

# Wait for last collection to finish
if collection_thread and collection_thread.is_alive():
    collection_thread.join()

# Final result
if stop_event.is_set() or loc_index < loc_threshold:
    print("❌ Not enough samples")
else:
    print("✅ All samples collected successfully.")

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 20 drops...

Current port set to 3
Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Current port set to 6
Rinsing complete.

Moving to location: well_plate_location at index 0
Movement commands rendered: 4
Moved safely To (X35.5, Y129.5, Z-12):  ['ok']

Current port set to 3
Starting fraction collection at well_plate_location (index 0)...

Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Fraction collection of well_plate_location (index 0) complete.

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
✅ All samples collected successfully.


In [5]:
# Simulate collection
loc_index = 0
loc_threshold = loc_index + number_of_samples*5
collection_thread = None

while not stop_event.is_set():
    try:
        sample = sample_queue.get(timeout=0.1)
        if collection_thread and collection_thread.is_alive():
            print("❌ Not enough time between samples — shutting down.")
            stop_event.set()
            break
        for i in range(5): # Collect 3 fractions per sample
            #collection_thread = threading.Thread(target=collect_sample_sim, args=(sample, loc_index, 3))
            collection_thread = threading.Thread(target=collector.collect_fraction, args=(50, loc, loc_index, 20)) #real collection
            collection_thread.start()
            collection_thread.join()
            loc_index += 1

    except queue.Empty:
        pass

    if loc_index >= loc_threshold:  # match number of samples
        break

# Wait for last collection to finish
if collection_thread and collection_thread.is_alive():
    collection_thread.join()

# Final result
if stop_event.is_set() or loc_index < loc_threshold:
    print("❌ Not enough samples")
else:
    print("✅ All samples collected successfully.")

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 20 drops...

Current port set to 3
Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Current port set to 6
Rinsing complete.

Moving to location: well_plate_location at index 0
Movement commands rendered: 4
Moved safely To (X35.5, Y132, Z-12):  ['ok']

Current port set to 3
Starting fraction collection at well_plate_location (index 0)...

Drop Counter (Drops) Started
ERROR: HID read timeout!
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
Timeout reached.
Failed to collect enough drops at well_plate_location (index 0)...

Current port set to 6
Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubi

In [5]:
stop_event.set()